In [241]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.dummy import DummyClassifier

In [7]:
raw_df = pd.read_csv("judge-1377884607_tweet_product_company.csv")

In [8]:
raw_df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [24]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [35]:
df = raw_df.rename({'is_there_an_emotion_directed_at_a_brand_or_product': 'sentiment'}, axis=1) \
           .drop('emotion_in_tweet_is_directed_at', axis=1)

In [36]:
df.head()

,tweet_text,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [37]:
sentiment_value_codes =   {'No emotion toward brand or product': 0,
                            'Positive emotion': 1,
                            'Negative emotion': -1,
                            "I can't tell": np.nan}

In [38]:
df['sentiment'] = df['sentiment'].map(sentiment_value_codes)

In [41]:
df = df.dropna().astype({'sentiment': 'int8'})

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8936 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  8936 non-null   object
 1   sentiment   8936 non-null   int8  
dtypes: int8(1), object(1)
memory usage: 148.4+ KB


## Text Pre-Processing

In [44]:
# nltk.download("stopwords")
# nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [177]:
def tweet_preprocessing(tweet, stops = stopwords.words('english'), lemmatizer = nltk.stem.WordNetLemmatizer()):
    '''
    Preprocesses a tweet for sentiment analysis using a given list of stop words and lemmatizer
    '''
        
    # Remove case
    lower_tweet = tweet.lower()

    # Tokenize on spaces and apostrophes
    token_tweet = lower_tweet.replace("'", " ").split(" ")
    
    # Define stopword list
    stops = stops + ['sxsw']
    
    # Define regex pattern
    pattern = re.compile('[^a-zA-Z]+') 
    
    # Create processed tweet
    proc_tweet = []
    
    for i in range(len(token_tweet)):
        clean_word = token_tweet[i]
        
        # Remove usernames
        if '@' in clean_word:
            clean_word = ""

        # Keep only characters
        clean_word = pattern.sub('', clean_word)

        # Remove stopwords and words 2 chars or less
        if (clean_word in stops) | (len(clean_word) <= 2):
            clean_word = ""
        
        # Lemmatize
        clean_word = lemmatizer.lemmatize(clean_word)
        
        # Replace original word with clean word if it's not empty
        if clean_word != "":
            proc_tweet.append(clean_word) 
    
    # Return string version of tweet
    return " ".join(proc_tweet)

In [178]:
# Get sample
sample_doc = df.iloc[0].tweet_text

print(tweet_preprocessing(sample_doc))

iphone hr tweeting riseaustin dead need upgrade plugin station


In [209]:
tweet_corpus = df['tweet_text'].apply(tweet_preprocessing)

## Count Vectorization

In [208]:
vec = CountVectorizer(min_df = 20)

In [210]:
X = vec.fit_transform(tweet_corpus)
y = df['sentiment']

In [211]:
vec_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
vec_df.head()

,able,access,aclu,action,actually,air,almost,already,also,always,...,work,working,world,would,wow,year,yes,yesterday,yet,zazzlesxsw
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modeling

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345, test_size=0.2)

In [240]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, random_state=6789, test_size=0.2)

In [234]:
scoring = ['precision_macro', 'recall_macro', 'accuracy']

## Baseline Dummy

In [235]:
dummy = DummyClassifier()

In [239]:
cv_dummy = cross_validate(dummy, X_train, y_train, cv=3, scoring = scoring)
print(cv_dummy)

{'fit_time': array([0.00099778, 0.00099683, 0.00099683]), 'score_time': array([0.00498581, 0.00199509, 0.0019958 ]), 'test_precision_macro': array([0.19960834, 0.19974822, 0.19983207]), 'test_recall_macro': array([0.33333333, 0.33333333, 0.33333333]), 'test_accuracy': array([0.59882501, 0.59924465, 0.59949622])}


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [244]:
dummy.fit(X_tr, y_tr)
y_val_pred = dummy.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        92
           0       0.59      1.00      0.74       845
           1       0.00      0.00      0.00       493

    accuracy                           0.59      1430
   macro avg       0.20      0.33      0.25      1430
weighted avg       0.35      0.59      0.44      1430



C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Multinomial Naive Bayes

In [237]:
mnb = MultinomialNB()

In [238]:
cv_mnb = cross_validate(mnb, X_train, y_train, cv=3, scoring = scoring)
print(cv_mnb)

{'fit_time': array([0.00199342, 0.00197434, 0.00199294]), 'score_time': array([0.0019927 , 0.00096488, 0.00099754]), 'test_precision_macro': array([0.53776132, 0.54903904, 0.57042006]), 'test_recall_macro': array([0.47012689, 0.47516616, 0.49078245]), 'test_accuracy': array([0.64582459, 0.64834243, 0.64609572])}


In [245]:
mnb.fit(X_tr, y_tr)
y_val_pred = mnb.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

          -1       0.43      0.22      0.29        92
           0       0.70      0.77      0.74       845
           1       0.58      0.54      0.56       493

    accuracy                           0.65      1430
   macro avg       0.57      0.51      0.53      1430
weighted avg       0.64      0.65      0.65      1430

